In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import ngsolve  # noqa: F401
else:
    try:
        import ngsolve
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/ngsolve-install.sh" -O "/tmp/ngsolve-install.sh" && bash "/tmp/ngsolve-install.sh"
        import ngsolve  # noqa: F401

In [ ]:
from netgen.meshing import Element1D, Mesh as NetgenMesh, MeshPoint, Pnt
from ngsolve import *
from ngsolve.webgui import Draw

In [ ]:
mesh = NetgenMesh(dim=1)
point_ids = [mesh.Add(MeshPoint(Pnt(i / 4, 0, 0))) for i in range(0, 4)]
[mesh.Add(Element1D([point_ids[i], point_ids[i + 1]], index=1)) for i in range(0, 3)]
mesh.SetMaterial(1, "interval")
mesh = Mesh(mesh)

In [ ]:
Draw(mesh)

In [ ]:
fes = H1(mesh, order=1)
assert fes.ndof == 4

In [ ]:
u = fes.TrialFunction()
v = fes.TestFunction()

In [ ]:
a = BilinearForm(fes, symmetric=True)
a += u * v * dx
a.Assemble()

In [ ]:
gff = GridFunction(fes)
for i in range(fes.ndof):
    gff.vec[i] = i + 1

In [ ]:
F = LinearForm(fes)
F += gff * v * dx
F.Assemble()

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * F.vec

In [ ]:
Draw(gfu)

In [ ]:
import numpy as np

In [ ]:
for i in range(fes.ndof):
    assert np.isclose(gfu.vec[i], i + 1)